# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

Importação das bibliotecas

In [29]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
import patsy

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

Aquisição dos dados em um df e seleção preliminar

In [30]:
df = pd.read_csv('previsao_de_renda.csv')
df_sig = df.iloc[:, 3:].copy() #removendo as colunas sem relação direta com previsão de renda
df_sig.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [31]:
df_sig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

Lista com nomes das variáveis, facilitando o input no patsy

In [32]:
list(df_sig.iloc[:, :-1].columns.values)

['sexo',
 'posse_de_veiculo',
 'posse_de_imovel',
 'qtd_filhos',
 'tipo_renda',
 'educacao',
 'estado_civil',
 'tipo_residencia',
 'idade',
 'tempo_emprego',
 'qt_pessoas_residencia']

Criação das partições do df e geração das *dummies*

In [33]:
y, X = patsy.dmatrices('''
                       np.log(renda) ~  sexo
                       + posse_de_veiculo
                       + posse_de_imovel
                       + qtd_filhos
                       + tipo_renda
                       + educacao
                       + estado_civil
                       + tipo_residencia
                       + idade
                       + tempo_emprego
                       + qt_pessoas_residencia''', data=df_sig)

Criação do modelo

In [34]:
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Sat, 15 Jun 2024   Prob (F-statistic):               0.00
Time:                        10:12:16   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            6.5847      0.235     28.006      0.000       6.124       7.046
sexo[T.M]                            0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]             0.0441      0.014      3.119      0.002       0.016       0.072
posse_de_imovel[T.True]              0.0829      0.014      5.926      0.000       0.055       0.110
tipo_renda[T.Bolsista]               0.2209      0.241      0.916      0.360      -0.252       0.694
tipo_renda[T.Empresário]             0.1551      0.015     10.387      0.000       0.126       0.184
tipo_renda[T.Pensionista]           -0.3087      0.241     -1.280      0.201      -0.782       0.164
tipo_renda[T.Servidor público]       0.0576      0.022      2.591      0.010       0.014       0.101
educacao[T.Pós graduação]            0.1071      0.159      0.673      0.501      -0.205       0.419
educacao[T.Secundário]              -0.0141      0.072     -0.196      0.844      -0.155       0.127
educacao[T.Superior completo]        0.0938      0.072      1.298      0.194      -0.048       0.235
educacao[T.Superior incompleto]     -0.0432      0.078     -0.555      0.579      -0.196       0.109
estado_civil[T.Separado]             0.3265      0.111      2.935      0.003       0.108       0.545
estado_civil[T.Solteiro]             0.2672      0.109      2.453      0.014       0.054       0.481
estado_civil[T.União]               -0.0343      0.025     -1.361      0.174      -0.084       0.015
estado_civil[T.Viúvo]                0.3689      0.116      3.191      0.001       0.142       0.596
tipo_residencia[T.Casa]             -0.0442      0.054     -0.815      0.415      -0.150       0.062
tipo_residencia[T.Com os pais]      -0.0258      0.060     -0.427      0.670      -0.144       0.093
tipo_residencia[T.Comunitário]      -0.1196      0.105     -1.135      0.256      -0.326       0.087
tipo_residencia[T.Estúdio]           0.0665      0.099      0.670      0.503      -0.128       0.261
tipo_residencia[T.Governamental]    -0.0571      0.066     -0.865      0.387      -0.186       0.072
qtd_filhos                          -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego                        0.0617      0.001     59.398      0.000       0.060       0.064
qt_pessoas_residencia                0.2926      0.108      2.716      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.858   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.651   Jarque-Bera (JB):                0.839
Skew:             

A significância estatística (P>|t|) parece fazer sentido. Ser empresário ou funcionário público parece prever renda, bem como idade, quantidade de filhos e tempo de emprego, além solitude (solteiros, viúvos e separados)

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

A variável menos significativa parece ser *educacao*, com os maiores *p-values* em todos os grupos

In [35]:
y, X = patsy.dmatrices('''
                       np.log(renda) ~  sexo
                       + posse_de_veiculo
                       + posse_de_imovel
                       + qtd_filhos
                       + tipo_renda
                       + estado_civil
                       + tipo_residencia
                       + idade
                       + tempo_emprego
                       + qt_pessoas_residencia''', data=df_sig)

In [36]:
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     340.0
Date:                Sat, 15 Jun 2024   Prob (F-statistic):               0.00
Time:                        10:12:16   Log-Likelihood:                -13601.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12406   BIC:                         2.740e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            6.6269      0.225     29.471      0.000       6.186       7.068
sexo[T.M]                            0.7800      0.015     53.208      0.000       0.751       0.809
posse_de_veiculo[T.True]             0.0533      0.014      3.769      0.000       0.026       0.081
posse_de_imovel[T.True]              0.0856      0.014      6.110      0.000       0.058       0.113
tipo_renda[T.Bolsista]               0.2988      0.242      1.237      0.216      -0.175       0.772
tipo_renda[T.Empresário]             0.1651      0.015     11.079      0.000       0.136       0.194
tipo_renda[T.Pensionista]           -0.2513      0.242     -1.039      0.299      -0.725       0.223
tipo_renda[T.Servidor público]       0.0754      0.022      3.400      0.001       0.032       0.119
estado_civil[T.Separado]             0.3238      0.112      2.903      0.004       0.105       0.542
estado_civil[T.Solteiro]             0.2616      0.109      2.396      0.017       0.048       0.476
estado_civil[T.União]               -0.0374      0.025     -1.481      0.139      -0.087       0.012
estado_civil[T.Viúvo]                0.3601      0.116      3.107      0.002       0.133       0.587
tipo_residencia[T.Casa]             -0.0369      0.054     -0.680      0.496      -0.143       0.069
tipo_residencia[T.Com os pais]      -0.0184      0.061     -0.304      0.761      -0.137       0.100
tipo_residencia[T.Comunitário]      -0.0772      0.105     -0.733      0.464      -0.284       0.129
tipo_residencia[T.Estúdio]           0.0855      0.100      0.859      0.390      -0.110       0.281
tipo_residencia[T.Governamental]    -0.0579      0.066     -0.875      0.382      -0.187       0.072
qtd_filhos                          -0.2567      0.108     -2.372      0.018      -0.469      -0.045
idade                                0.0049      0.001      6.185      0.000       0.003       0.006
tempo_emprego                        0.0617      0.001     59.327      0.000       0.060       0.064
qt_pessoas_residencia                0.2846      0.108      2.634      0.008       0.073       0.496
==============================================================================
Omnibus:                        1.217   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.544   Jarque-Bera (JB):                1.187
Skew:                           0.021   Prob(JB):                        0.553
Kurtosis:                       3.023   Cond. No.                     2.15e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.15e+03. This might indicate that there are
stron

O modelo parece ter piorado, pois o valor de *R²-ajustado* diminuiu e *AIC* aumentou, porém todos os *p-value* das covariáveis restantes parecem ter diminuido significativamente 

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [37]:
y, X = patsy.dmatrices('''
                       np.log(renda) ~  sexo
                       + posse_de_veiculo
                       + posse_de_imovel
                       + qtd_filhos
                       + idade
                       + tempo_emprego
                       + qt_pessoas_residencia''', data=df_sig)

In [38]:
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     940.8
Date:                Sat, 15 Jun 2024   Prob (F-statistic):               0.00
Time:                        10:12:16   Log-Likelihood:                -13672.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12419   BIC:                         2.742e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    7.2377      0.043    166.875      0.000       7.153       7.323
sexo[T.M]                    0.7694      0.015     52.676      0.000       0.741       0.798
posse_de_veiculo[T.True]     0.0569      0.014      4.022      0.000       0.029       0.085
posse_de_imovel[T.True]      0.0866      0.014      6.275      0.000       0.060       0.114
qtd_filhos                   0.0338      0.019      1.735      0.083      -0.004       0.072
idade                        0.0049      0.001      6.408      0.000       0.003       0.006
tempo_emprego                0.0610      0.001     59.075      0.000       0.059       0.063
qt_pessoas_residencia       -0.0092      0.016     -0.566      0.572      -0.041       0.023
==============================================================================
Omnibus:                        1.240   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.538   Jarque-Bera (JB):                1.213
Skew:                           0.022   Prob(JB):                        0.545
Kurtosis:                       3.019   Cond. No.                         300.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
y, X = patsy.dmatrices('''
                       np.log(renda) ~  sexo
                       + posse_de_veiculo
                       + posse_de_imovel
                       + idade
                       + tempo_emprego''', data=df_sig)

In [41]:
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1315.
Date:                Sat, 15 Jun 2024   Prob (F-statistic):               0.00
Time:                        10:12:50   Log-Likelihood:                -13676.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12421   BIC:                         2.741e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    7.2550      0.032    228.981      0.000       7.193       7.317
sexo[T.M]                    0.7700      0.015     52.851      0.000       0.741       0.799
posse_de_veiculo[T.True]     0.0579      0.014      4.116      0.000       0.030       0.085
posse_de_imovel[T.True]      0.0880      0.014      6.379      0.000       0.061       0.115
idade                        0.0044      0.001      5.871      0.000       0.003       0.006
tempo_emprego                0.0611      0.001     59.199      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.248   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.536   Jarque-Bera (JB):                1.221
Skew:                           0.022   Prob(JB):                        0.543
Kurtosis:                       3.019   Cond. No.                         209.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Retirei todas as variáveis, me baseando nos *p-values* maiores que 5%. Comparando os valores de *R²-ajustado* e *AIC* o modelo parece ter piorado, pois o primeiro diminuiu e o segundo aumentou, mesmo com as variáveis restantes sendo significativas ao modelo.